## **Órbita de Asteroide**

In [ ]:
!pip install rebound

In [ ]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter

In [ ]:
sim = rebound.Simulation()

# Unidades de la simulación
sim.units = ('yr', 'AU', 'Msun')

# Agregar partículas (Sol, Tierra y un pequeño asteroide)
sim.add(m=1.0)  #Sol, masa = 1.0 (en masas solares)
sim.add(m=3e-6, a=1.0, e=0.0167)  # Tierra, a = 1.0 AU, e = 0.0167 (parámetros orbitales)
sim.add(m=1e-10, a=1.5, e=0.1)  # Asteroide pequeño, con cierta excentricidad

# Integrador y otras propiedades de la simulación
sim.integrator = "whfast"
sim.dt = 0.01  # Paso (en years)

times = np.linspace(0, 10, 1000)  # Simular por 10 years
positions = np.zeros((len(times), 3, 3))  # Guardar posiciones de los astros

for i, t in enumerate(times):
    sim.integrate(t)
    positions[i] = np.array([[p.x, p.y, p.z] for p in sim.particles])  # Posiciones x, y, z

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
plt.plot(positions[:, 1, 0], positions[:, 1, 1], label="Tierra")
plt.plot(positions[:, 2, 0], positions[:, 2, 1], label="Asteroide")
plt.scatter(0, 0, color='orange', label="Sol", s=100)
plt.xlabel("x [AU]")
plt.ylabel("y [AU]")
plt.legend()
plt.title("Órbitas")
plt.show()

In [ ]:
# Unidades de la simulación
sim.units = ('yr', 'AU', 'Msun')

# Agregar partículas (Sol, Tierra y un pequeño asteroide)
sim.add(m=1.0)  #Sol, masa = 1.0 (en masas solares)
sim.add(m=3e-6, a=1.0, e=0.0167)  # Tierra, a = 1.0 AU, e = 0.0167 (parámetros orbitales)
sim.add(m=1e-10, a=1.5, e=0.1)  # Asteroide pequeño, con cierta excentricidad

# Integrador y otras propiedades de la simulación
sim.integrator = "whfast"
sim.dt = 0.01  # Paso (en years)

times = np.linspace(0, 10, 1000)  # Simular por 10 years
positions = np.zeros((len(times), 3, 3))  # Guardar posiciones de los astros

for i, t in enumerate(times):
    sim.integrate(t)
    positions[i] = np.array([[p.x, p.y, p.z] for p in sim.particles])  # Posiciones x, y, z


fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

sun, = ax.plot(0, 0, 'o', color='orange', markersize=10, label="Sun")

earth_orbit, = ax.plot([], [], '-', color='blue', lw=1, label="Earth Orbit")
asteroid_orbit, = ax.plot([], [], '-', color='green', lw=1, label="Asteroid Orbit")
earth, = ax.plot([], [], 'o', color='blue', markersize=5, label="Earth")
asteroid, = ax.plot([], [], 'o', color='green', markersize=5, label="Asteroid")

def init():
    earth_orbit.set_data([], [])
    asteroid_orbit.set_data([], [])
    earth.set_data([], [])
    asteroid.set_data([], [])
    return earth_orbit, asteroid_orbit, earth, asteroid

def update(frame):
    earth_orbit.set_data(positions[:frame, 1, 0], positions[:frame, 1, 1])
    asteroid_orbit.set_data(positions[:frame, 2, 0], positions[:frame, 2, 1])
    earth.set_data(positions[frame, 1, 0], positions[frame, 1, 1])
    asteroid.set_data(positions[frame, 2, 0], positions[frame, 2, 1])
    return earth_orbit, asteroid_orbit, earth, asteroid

# Creemos la animación
ani = FuncAnimation(fig, update, frames=len(times), init_func=init, blit=True, interval=20)

# Guardarla como GIF
ani.save("/content/orbit_animation.gif", writer=PillowWriter(fps=30))

# Mostrar animación
from IPython.display import Image
Image(filename="/content/orbit_animation.gif")

## **Galaxia**

In [ ]:
!pip install yt

In [ ]:
!wget http://yt-project.org/data/IsolatedGalaxy.tar.gz

In [ ]:
!tar -xzf IsolatedGalaxy.tar.gz.1

In [ ]:
import yt

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

p_dm = yt.ParticleProjectionPlot(ds, "z", ("all", "particle_mass"))
p_dm.set_cmap(("all", "particle_mass"), "inferno")
p_dm.annotate_title("Dark Matter Mass Projection")
p_dm.show()

In [ ]:
p_dm.set_unit(("all", "particle_mass"), "Msun")  # Set units to solar masses
p_dm.set_log(("all", "particle_mass"), True)  # Use a logarithmic scale for mass projection
p_dm.set_zlim(("all", "particle_mass"), 1e5, 1e9)  # Set limits for the color scale
p_dm.show()

In [ ]:
print(ds.field_list)

In [ ]:
s = yt.SlicePlot(ds, "z", ("gas", "density"))
s.set_cmap(("gas", "density"), "magma")
s.show()

In [ ]:
p_temp = yt.ProjectionPlot(ds, "z", ("gas", "temperature"))
p_temp.set_cmap("temperature", "coolwarm")
p_temp.show()

In [ ]:
p_velocity = yt.ProjectionPlot(ds, "z", ("gas", "velocity_magnitude"))
p_velocity.show()

## **Estructura a gran escala**

In [ ]:
!pip install scipy
!pip install pillow

In [ ]:
n_particles = 10000  # Número de partículas
box_size = 100.0    # Tamaño de la caja
n_steps = 300       # Número de pasos de la simulación

x = np.random.uniform(0, box_size, n_particles)
y = np.random.uniform(0, box_size, n_particles)
z = np.random.uniform(0, box_size, n_particles)

# Pequeñas perturbaciones de velocidad (aleatorías) para comenzar la formación de estructura
vx = np.random.normal(0, 0.05, n_particles)
vy = np.random.normal(0, 0.05, n_particles)
vz = np.random.normal(0, 0.05, n_particles)

# Loop de simulación
positions = []
for step in range(n_steps):
    x += vx
    y += vy
    z += vz

    # Condiciones de frontera
    x = np.mod(x, box_size)
    y = np.mod(y, box_size)
    z = np.mod(z, box_size)

    positions.append(np.column_stack([x, y, z]))

positions = np.array(positions)

fig, ax = plt.subplots(figsize=(8, 8))
scat = ax.scatter(positions[0][:, 0], positions[0][:, 1], s=5, alpha=0.7)
ax.set_xlim([0, box_size])
ax.set_ylim([0, box_size])

def update(frame):
    scat.set_offsets(positions[frame][:, :2])  # Actualizar posiciones de las partículas
    return scat,

# Crear la animación
ani = FuncAnimation(fig, update, frames=n_steps, interval=100, blit=True)

# Guardemola como GIF
ani.save("large_scale_structure.gif", writer=PillowWriter(fps=10))

from IPython.display import Image
Image(filename="large_scale_structure.gif")